In [ ]:
!pip install jieba
import jieba
import pandas as pd
from sklearn.cross_validation import train_test_split
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
data = pd.read_csv('./data/session.csv',encoding='gb18030')

In [ ]:
data.head()

In [ ]:
data['咨询类型'].unique()

In [ ]:
basic_type = data['咨询类型'].str.split('->', n=1, expand=True)

In [ ]:
data['基本咨询类型'] = basic_type[0]

In [ ]:
data['基本咨询类型'].unique()

In [ ]:
class_mapping = {label:'__label__'+str(idx) for idx,label in enumerate(set(data['基本咨询类型']))}
data['label'] = data['基本咨询类型'].map(class_mapping)

In [ ]:
class_mapping

In [ ]:
data['label'].unique()

In [ ]:
len(data['label'].unique())

In [ ]:
details = data['详情'].str.split('\n', expand=True)

In [ ]:
details.head()

In [ ]:
def get_first_sentence(sentences):
    first_sentence = ''
    for i in range(len(sentences)):
        if sentences[i] is not None and '2019-05-23' in sentences[i] and '蜜芽' not in sentences[i]:
            if len(sentences[i+1]) > 3 and '人工客服' not in sentences[i+1] and '在线咨询' not in sentences[i+1] and '发送图片' not in sentences[i+1]:
                first_sentence += sentences[i+1]+' '
                # break  # only first sentence
    seg_list = jieba.cut(first_sentence, cut_all=False)
    first_sentence = ' '.join(seg_list)
    return first_sentence

In [ ]:
first_sentences = details.apply(lambda row: get_first_sentence(row), axis=1)

In [ ]:
first_sentences.head()

In [ ]:
data['first_sentence'] = first_sentences

In [ ]:
data = data[data['first_sentence'] != '']
train, dev = train_test_split(data, test_size=0.1, random_state=2019)

In [ ]:
print(train.shape, dev.shape)  # test.shape, 

In [ ]:
train[['label', 'first_sentence']].to_csv('data/mytask/train.tsv', index=False, header=False, sep='\t')
#test[['label', 'first_sentence']].to_csv('data/mytask/test.tsv', index=False, header=False, sep='\t')
dev[['label', 'first_sentence']].to_csv('data/mytask/dev.tsv', index=False, header=False, sep='\t')

In [ ]:
!aws s3 cp data/mytask/train.tsv s3://app-mia/mytask/
#!aws s3 cp data/mytask/test.tsv s3://app-mia/mytask/
!aws s3 cp data/mytask/dev.tsv s3://app-mia/mytask/